In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import time
import random

In [4]:
# 2x2 cube
solved_state = np.array([0,1,2,3,4,5,6,7,0,0,0,0,0,0,0,0])

move_map = {
    "R": np.array([0, 2, 5, 3, 4, 6, 1, 7, 0, 1, 2, 0, 0, 1, 2, 0]),
    "U": np.array([3, 0, 1, 2, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0]),
    "F": np.array([0, 1, 3, 4, 5, 2, 6, 7, 0, 0, 1, 2, 1, 2, 0, 0]),
    "R'": np.array([0, 6, 1, 3, 4, 2, 5, 7, 0, 1, 2, 0, 0, 1, 2, 0]),
    "U'": np.array([1, 2, 3, 0, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0]),
    "F'": np.array([0, 1, 5, 2, 3, 4, 6, 7, 0, 0, 1, 2, 1, 2, 0, 0]),
    "R2": np.array([0, 5, 6, 3, 4, 1, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0]),
    "U2": np.array([2, 3, 0, 1, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0]),
    "F2": np.array([0, 1, 4, 5, 2, 3, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0]),
}

moves = ["U", "U2", "U'", "R", "R2", "R'", "F", "F2", "F'"]

@njit
def apply_move(cube, move_array):
    cube2 = cube.copy()
    # Orientation
    cube[8:16] = (cube2[(move_array[:8]+8)] + move_array[8:16]) % 3 

    # rewrite the orientation
    cube[:8] = cube[move_array[:8]]

    return cube

def apply_alg(cube, alg):
    for move in alg.split(" "):
        cube = apply_move(cube, move_map[move])

    return cube

def get_cube(scramble = ""):
    if scramble == "":
        return np.array([0,1,2,3,4,5,6,7,0,0,0,0,0,0,0,0])
    else:
        cube = np.array([0,1,2,3,4,5,6,7,0,0,0,0,0,0,0,0])
        cube = apply_alg(cube, scramble)
        return cube

cube = get_cube("")

move_array = move_map["R"]
cube = apply_move(cube, move_array)

def inv(solution):
    return " ".join(
        (s[:-1] if "'" in s else s + "'") if "2" not in s else s
        for s in solution.split()[::-1]
    )

In [5]:
# we wanna make a solver
# id 7 is never used

@njit
def get_id_from_state(cube):
    ID0 = 0
    for i in range(6):
        ID0 += cube[i] * 7**i
    
    ID1 = 0
    for i in range(6):
        ID1 += cube[i+8] * 3**(i)
    
    return ID0*3**6 + ID1

cube = get_cube("R U R' U' R' F R2 U' R' U' R U R' F'")
cube = get_cube(inv("F R2 U2 F U2 R F' R"))
get_id_from_state(cube)


68525328

In [6]:
@njit
def _inc(ids):
    for i in range(len(ids)):
        ids[i]+=1
        ids[i]%=9
        if ids[i]:
            break
    return ids

@njit
def _is_valid(length,ids):
    for i in range(length-1):
        if ids[i]//3==ids[i+1]//3:
            return False
    return True      

@njit
def _increment(length,ids):
    ids = _inc(ids)
    while not _is_valid(length,ids):
        ids = _inc(ids)
    return ids


class alg_index:
    def __init__(self, depth):
        self.depth = depth
        self.alg = np.array([0,3,6]*(depth//3+1))[:depth]

    def increment(self):
        self.alg = _increment(self.depth,self.alg)

    def __str__(self):
        return " ".join(moves[i] for i in self.alg)
    

ai = alg_index(3)

def gen_all_algs(depth, print_progress = False):
    all_algs = []
    for i in range(1, depth+1):
        if print_progress:
            print(f"Genning algs of length {i}...")
        ai = alg_index(i)
        start_alg = str(ai)
        ai.increment()
        while str(ai) != start_alg:
            all_algs.append(str(ai))
            ai.increment()

    return all_algs

algs = gen_all_algs(3)

In [7]:
def gen_table(depth, print_progress = False):
    algs = gen_all_algs(depth, print_progress)
    table = {}
    if print_progress:
        print("Generating table...")
    for alg in algs:
        cube = get_cube(alg)
        ID = get_id_from_state(cube)
        if ID not in table:
            table[ID] = inv(alg)

    return table

table = gen_table(8, True)

Genning algs of length 1...
Genning algs of length 2...
Genning algs of length 3...
Genning algs of length 4...
Genning algs of length 5...
Genning algs of length 6...
Genning algs of length 7...
Genning algs of length 8...
Generating table...


In [8]:
def solver(cube, search_algs, table):
    ID = get_id_from_state(cube)
    if ID in table:
        return table[ID]
    
    for alg in search_algs:
        cube = apply_alg(cube, alg)
        ID = get_id_from_state(cube)
        if ID in table:
            return alg + " " + table[ID]
        cube = apply_alg(cube, inv(alg))
        
    return "No solution found"

search_algs = gen_all_algs(3)

cube = get_cube("R U R' U' R' F R2 U' R' U' R U R' F'")
solver(cube, search_algs, table)

"F2 R U F' R2 F R' F U R2"

In [9]:
t = time.time()
for i in range(10000):
    cube = get_cube("R U R' U' R' F R2 U' R' U' R U R' F'")
    solver(cube, search_algs, table)
print(time.time()-t)

0.5085499286651611


In [14]:
# load scrambles.csv - it's a csv file with a bunch of scrambles
# there are many columns, we only want the first one, from the 1st row (not the header)
# there are ; in the csv file, we want to split by that

import csv

scrambles = []
with open("scrambles.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        scrambles.append(row[0].split(";")[0])

scrambles = scrambles[1:]

In [22]:
t = time.time()
lengths = [0]*12
for i, scramble in enumerate(scrambles):
    if i % 3000 == 0:
        print(f"{i}/{len(scrambles)} done in {time.time()-t} seconds")
    cube = get_cube(scramble)
    sol = solver(cube, search_algs, table)
    lengths[len(sol.split(" "))] += 1

print(time.time()-t)
    

0/301183 done in 0.0 seconds
3000/301183 done in 0.08676266670227051 seconds
6000/301183 done in 0.17235636711120605 seconds
9000/301183 done in 0.24095535278320312 seconds
12000/301183 done in 0.31084465980529785 seconds
15000/301183 done in 0.37687206268310547 seconds
18000/301183 done in 0.4420146942138672 seconds
21000/301183 done in 0.5106029510498047 seconds
24000/301183 done in 0.592308521270752 seconds
27000/301183 done in 0.6871230602264404 seconds
30000/301183 done in 0.7556936740875244 seconds
33000/301183 done in 0.8212594985961914 seconds
36000/301183 done in 0.8919789791107178 seconds
39000/301183 done in 0.9621076583862305 seconds
42000/301183 done in 1.032104730606079 seconds
45000/301183 done in 1.0998268127441406 seconds
48000/301183 done in 1.1929802894592285 seconds
51000/301183 done in 1.2931387424468994 seconds
54000/301183 done in 1.379338264465332 seconds
57000/301183 done in 1.4617602825164795 seconds
60000/301183 done in 1.5354280471801758 seconds
63000/301183

[0, 0, 0, 0, 151, 766, 4032, 18511, 71617, 148265, 57631, 210]